In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
import shutil
from PIL import Image
import skimage
from skimage.io import imread
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

In [2]:
path=r'C:\Users\Biswarup Mistry\Desktop\USC_SIPI_Databases' #USC_SIPI_Database directory [To Change 1]
dir_list=os.listdir(path)
print(len(dir_list))

4


In [3]:
directory = "Training Set"
directory1 = "Test Set"

In [4]:
for item in dir_list :
    if item != "misc" :
        p=str(path+'\\'+str(item))
        dir_lst=os.listdir(p)
        #a=random.choices(dir_lst, k=3)  #Choose multiple random items from a list, set, or any data structure.
        i=1
        cal=math.floor((len(dir_lst)*90)/100)
        cal1=len(dir_lst)-cal
        #directory = "Training Set"
        parent_dir = p
        path1 = os.path.join(parent_dir, directory)
        if os.path.exists(path1)==False :
            os.mkdir(path1) #making the 'Training Set' Directory
            new_path=str(parent_dir+'\\'+directory)

            while i<=cal :
                a=random.choices(os.listdir(p)) #randomly choosen 80% training data set of current dataset & gives a list
                for image in a :
                    if image != directory :
                        old_path=p+'\\'+image
                        n_path=new_path+'\\'+image
                        #try :
                        shutil.move(old_path, n_path) #to move the files
                        #except FileNotFoundError :
                            #print('Error : File not Found for Image = ',image)
                        i+=1

            #directory1 = "Test Set"
            #if os.path.exists(path+'\\'+directory)!=True :
            parent_dir = p
            path1 = os.path.join(parent_dir, directory1)
            os.mkdir(path1) #making the 'Training Set' Directory
            new_path=str(parent_dir+'\\'+directory1)
            a=os.listdir(p)
            for item in a:
                if (item != directory) and (item != directory1) :
                    old_path=p+'\\'+item
                    n_path=new_path+'\\'+item
                    shutil.move(old_path, n_path) 


In [5]:
def get_Noise(image, prob):
    #'''
    #Add salt and pepper noise to image
    #prob: Probability of the noise
    #'''
    output = image.copy()
    if len(image.shape) == 2:
        black = 0
        white = 255            
    else:
        colorspace = image.shape[2]
        if colorspace == 3:  # RGB
            black = np.array([0, 0, 0], dtype='uint8')
            white = np.array([255, 255, 255], dtype='uint8')
        else:  # RGBA
            black = np.array([0, 0, 0, 255], dtype='uint8')
            white = np.array([255, 255, 255, 255], dtype='uint8')
    probs = np.random.random(output.shape[:2])
    output[probs < (prob / 2)] = black
    output[probs > 1 - (prob / 2)] = white
    return output

In [6]:
def SSIMVsNoiseDensity(dirc_list,dataframe): #getting corresponding mse(s) for given noise density 'd'
    r=255.0
    psnr='PSNR_of_90_percent_Data.csv'
    ssim1='SSIM_of_90_percent_Data.csv'
    #dataframe = pd.DataFrame()
    for item in os.listdir(dirc_list) :
        if item!=psnr and item!=ssim1:
            #k=str(path+str(item))
            k=str(dirc_list+'\\'+item)
            img = imread(k)
            #x=[]
            #y=[]
            df=np.array([])
            #i=iteration
            #d=0.1
            #while(i>0 and d<=0.3):
            #D=[0.1,0.2,0.3]
            D=[0.1,0.25,0.3,0.35,0.4]
            for d in D:
                #img=np.squeeze(img)



                noise_img=get_Noise(img,d) #getting noisy image

                #---------------------- (conversion for SSIM)
                if len(img.shape)!=2 : #if image is not gray, i.e., RGB
                    im = rgb2gray(img)
                    noise_im=rgb2gray(noise_img)
                else : #if image is Gray already
                    im=img
                    noise_im=noise_img
                #---------------------- (conversion for SSIM)

                SSIM = ssim(im,noise_im) #Obtaining the value of SSIM b/t two Images(Noisy and Original)

                #x.append(d)
                #y.append(SSIM)
                df=np.append(df,SSIM)
            dataframe[item] = df


In [7]:
def createFolder(path):
    count=0
    count1=0
    dir_list=os.listdir(path)
    parent_dir=path
    dir_list=os.listdir(path)
    check1=str(path+'\\'+'256')
    check2=str(path+'\\'+'512')
    check3=str(path+'\\'+'1024')
    
    isExist1=os.path.exists(check1)
    isExist2=os.path.exists(check2)
    isExist3=os.path.exists(check3)
    
    if isExist1==isExist2==isExist3==False:
        for item in dir_list:
            if (item!=check1) and (item!=check2) and (item!=check3):
                k=str(path+'\\'+item)
                img = Image.open(k)
                w, h= img.size
                img.close()
                file_path=str(path+'\\'+str(w))
                isExist=os.path.exists(file_path)
                if isExist==False:
                    directory=str(w)
                    path1 = os.path.join(parent_dir, directory)
                    os.mkdir(path1)
                    count1+=1
    else:
        print('Folders already exist!')
    print('Created Folder : ',count1)
    

In [8]:

def moveImage(path):
    psnr='PSNR_of_90_percent_Data.csv'
    ssim1='SSIM_of_90_percent_Data.csv'
    count=0
    dir_list=os.listdir(path)
    check1=str(path+'\\'+'256')
    check2=str(path+'\\'+'512')
    check3=str(path+'\\'+'1024')
    #print(len(dir_list))
    if len(dir_list)>3:
        for item in dir_list:
            k=str(path+'\\'+item) 
            isdir = os.path.isdir(k)
            #if isdir== False and item!=psnr and item!=ssim1  :
            if item.lower().endswith(('.png', '.jpg', '.jpeg','.tiff')) :
                img = Image.open(k)
                w, h= img.size
                file_path=str(path+'\\'+str(w))
                old_path=str(path+'\\'+str(item))
                n_path=str(file_path+'\\'+str(item))
                img.close()
                shutil.move(old_path, n_path)
                count+=1
        
        print('Image(s) moved : ',count)
    elif len(dir_list)==3:
        print('Nothing to Move!')


In [9]:
psnr='PSNR_of_90_percent_Data.csv'
ssim1='SSIM_of_90_percent_Data.csv'
for item in dir_list :
    if item != "misc" :
        p=str(path+'\\'+str(item))
        dir_lst=os.listdir(p)
        for item in dir_lst:
            if item != directory1 :
                new_path=str(p+'\\'+str(item))
                createFolder(new_path)
                moveImage(new_path)
                for item in os.listdir(new_path):
                    p1=str(new_path+'\\'+str(item))
                    if os.path.exists(str(p1+'\\'+ssim1))==False:
                        dataframe = pd.DataFrame()
                        SSIMVsNoiseDensity(p1,dataframe)
                        #for item in os.listdir(new_path): #under Training Set
                        #    k=str(new_path+'\\'+str(item))
                        #    getPSNR(image)
                        dataframe.to_csv(p1+'\\'+'SSIM_of_90_percent_Data.csv', index=False) 
                    else :
                        print(psnr,' Exists!!')
            #print(dir_lst)

Folders already exist!
Created Folder :  0
Folders already exist!
Created Folder :  0
Folders already exist!
Created Folder :  0


In [10]:
for item in os.listdir(path):
    if item != "misc":
        p=str(path+'\\'+str(item))
        
        for item in os.listdir(p):
            if item == directory :
                print(item)
                new_path=str(p+'\\'+directory) #under training set
                for item in os.listdir(new_path):
                    k=str(new_path+'\\'+str(item))
                    print(k)
                    try:
                        data=pd.read_csv(k+'\\'+'SSIM_of_90_percent_Data.csv')
                        print(data)
                    except:
                        print("SSIM_of_90_percent_Data.csv not Exists!!")


Training Set
C:\Users\Biswarup Mistry\Desktop\USC_SIPI_Databases\aerials\Training Set\1024
   2.2.01.tiff  2.2.02.tiff  2.2.05.tiff  2.2.06.tiff  2.2.07.tiff  \
0     0.376562     0.194313     0.332480     0.187424     0.212519   
1     0.175898     0.073501     0.145331     0.069142     0.078679   
2     0.144872     0.059801     0.120123     0.056689     0.064385   
3     0.122358     0.050481     0.100225     0.047490     0.053989   
4     0.103290     0.043612     0.084891     0.040863     0.046066   

   2.2.08.tiff  2.2.09.tiff  2.2.10.tiff  2.2.11.tiff  2.2.12.tiff  ...  \
0     0.373881     0.226117     0.211969     0.262024     0.247347  ...   
1     0.173903     0.088942     0.080717     0.107036     0.102187  ...   
2     0.144065     0.072473     0.066801     0.087384     0.083685  ...   
3     0.120805     0.060599     0.055303     0.072794     0.069728  ...   
4     0.101580     0.052023     0.047606     0.061353     0.059128  ...   

   2.2.15.tiff  2.2.16.tiff  2.2.17.t